## 1. 데이터를 웹에서 받아오기

In [ ]:
# -*- coding: utf-8 -*-
from urllib.request import urlretrieve
import zipfile
import os

if __name__ == "__main__":
    
    # 데이터 위치 지정
    data_dir = './data/original'

    # 폴더 없으면 만들기
    os.makedirs(data_dir, exist_ok=True)

    # data 들어갈 위치지정, zip파일지정
    data_archive = os.path.join(data_dir, 'jsut-data.zip')
    print('download jsut-data start')
    urlretrieve('http://ss-takashi.sakura.ne.jp/corpus/jsut_ver1.1.zip', 
                data_archive)
    print('download jsut-data finished')

    # zip파일 풀기
    print('extract jsut-data start')
    with zipfile.ZipFile(data_archive) as data_zip:
        data_zip.extractall(data_dir)
    print('extract jsut-data finished')

    # zip파일 삭제
    os.remove(data_archive)

    # jsut 코퍼스 라벨 데이터 다운로드
    label_archive = os.path.join(data_dir, 'jsut-label.zip')
    print('download jsut-label start')
    urlretrieve('https://github.com/sarulab-speech/jsut-label/archive/master.zip',
                label_archive)
    print('download jsut-label finished')

    # 다운로드한 데이터 배포
    print('extract jsut-label start')
    with zipfile.ZipFile(label_archive) as label_zip:
        label_zip.extractall(data_dir)
    print('extract jsut-label finished')

    # zip파일 삭제
    os.remove(label_archive)

    print('all processes finished')



## 2. wav 파일을 48khz->16khz로 변경

In [ ]:
# -*- coding: utf-8 -*-
import sox
import os
if __name__ == "__main__":
    
    # wav 파일이 확장된 디렉토리
    original_wav_dir = './data/original/jsut_ver1.1/basic5000/wav/'

    # 포맷 변환한 wav 파일을 출력하는 디렉토리
    out_wav_dir = './data/wav/'

    # wav 데이터 목록을 저장하는 디렉토리
    out_scp_dir = './data/label/all/'

    # 출력 디렉토리가 없으면 작성
    os.makedirs(out_wav_dir, exist_ok=True)
    os.makedirs(out_scp_dir, exist_ok=True)

    # sox로 음성 변환 클래스 호출
    tfm = sox.Transformer()
    # 샘플링 주파수를 16000Hz로 변환하도록 설정
    tfm.convert(samplerate=16000)

    # wav 데이터의 리스트 파일을 write 모드로 열어, 이후의 처리를 실시한다
    with open(os.path.join(out_scp_dir, 'wav.scp'), mode='w') as scp_file:
        # BASIC5000_0001.wav ~ BASIC5000_5000.wav 처리를 반복 실행
        for i in range(5000):
            filename = 'BASIC5000_%04d' % (i+1)
            # 변환 소스의 원본 데이터 (48000Hz) 파일 이름
            wav_path_in = os.path.join(original_wav_dir, filename+'.wav')
            # 변환 후 데이터 (16000Hz)의 저장 파일 이름
            wav_path_out = os.path.join(out_wav_dir, filename+'.wav')

            print(wav_path_in)
            # 파일이 없으면 오류
            if not os.path.exists(wav_path_in):
                print('Error: Not found %s' % (wav_path_in))
                exit()

            # 샘플링 주파수 변환 및 저장 수행
            tfm.build_file(input_filepath=wav_path_in, 
                           output_filepath=wav_path_out)

            # wav 파일 목록 쓰기
            scp_file.write('%s %s\n' % 
                           (filename, os.path.abspath(wav_path_out)))
        


# SOX 오류가 나서 찾아보았음

### 1. sox conda로 최신버전 깔기
https://anaconda.org/conda-forge/sox

### 2. libvorbis 깔기
https://anaconda.org/conda-forge/libvorbis/

## 3. 2번째 전처리->yaml파일로 데이터 label.txt 만들기

In [ ]:

# -*- coding: utf-8 -*-
import yaml
import os

if __name__ == "__main__":
    
    # 다운로드한 라벨 데이터(yaml 형식)
    original_label = \
      './data/original/jsut-label-master/text_kana/basic5000.yaml'

    
    # 라벨 목록을 저장할 위치
    out_label_dir = './data/label/all'

    # 출력 디렉토리가 없으면 작성
    os.makedirs(out_label_dir, exist_ok=True)

    # 라벨 데이터 로드
    with open(original_label, mode='r',encoding='UTF8') as yamlfile: #UTF8 지정을 해줘야함
        label_info = yaml.safe_load(yamlfile)

    # char / kana / 음소 라벨 파일을 write 모드로 열기, 얘들도 UTF8로 열어야함
    with open(os.path.join(out_label_dir, 'text_char'), 
              mode='w',encoding='UTF8') as label_char, \
              open(os.path.join(out_label_dir, 'text_kana'), 
              mode='w',encoding='UTF8') as label_kana, \
              open(os.path.join(out_label_dir, 'text_phone'), 
              mode='w',encoding='UTF8') as label_phone:
        # BASIC5000_0001 ~ BASIC5000_5000 처리를 반복실행
        for i in range(5000):
            # 발화ID
            filename = 'BASIC5000_%04d' % (i+1)
            
            # 발화 ID가 label_info에 포함되지 않은 경우 오류
            if not filename in label_info:
                print('Error: %s is not in %s' % (filename, original_label))
                exit()

            # Chars 라벨 정보 획득
            chars = label_info[filename]['text_level2']
            # '、'と'。'を除去
            chars = chars.replace('、', '')
            chars = chars.replace('。', '')

            # kana 라벨 정보 획득
            kanas = label_info[filename]['kana_level3']
            # '、'を除去
            kanas = kanas.replace('、', '')

            # 음소 라벨 정보 얻기
            phones = label_info[filename]['phone_level3']

            # char 라벨 파일에, 1 문자씩 스페이스 단락으로 기입
            # ' '.join (list)는 목록의 각 요소에 공백을 넣고 한 문장으로 만듬
            label_char.write('%s %s\n' % (filename, ' '.join(chars)))

            # kana 라벨 파일에 한 문자씩 공백으로 구분
            label_kana.write('%s %s\n' % (filename, ' '.join(kanas)))

            # 음소 라벨은 '-'를 공백으로 대체하여 사용
            label_phone.write('%s %s\n' % (filename, phones.replace('-',' ')))



## 4. 데이터를 학습/개발/평가 데이터 세트로 나눔 

In [ ]:
# -*- coding: utf-8 -*-
# 데이터 목록을 학습 / 개발 / 평가를위한 데이터 세트로 나눔
# 다음과 같이 분할
# BASIC5000_0001~0250 : 평가 데이터
# BASIC5000_0251~0500 : 개발 데이터
# BASIC5000_0501~1500 : 학습 데이터(소)
# BASIC5000_0501~5000 : 학습 데이터(대)
import os


if __name__ == "__main__":
    
    # 모든 데이터가 기술된 목록의 위치
    all_dir = './data/label/all/'

    # 평가(test) 데이터가 기술된 리스트의 출력처
    out_eval_dir = './data/label/test/'
    # 개발(검증) 데이터가 기술된 리스트의 출력처
    out_dev_dir = './data/label/dev'
    # 학습 데이터(소)가 기술된 리스트의 출력처
    out_train_small_dir = './data/label/train_small/'
    # 학습 데이터(대)가 기술된 리스트의 출력처
    out_train_large_dir = './data/label/train_large/'

    # 각 출력 디렉토리가 존재하지 않는 경우 작성
    for out_dir in [out_eval_dir, out_dev_dir, 
                    out_train_small_dir, out_train_large_dir]:
        os.makedirs(out_dir, exist_ok=True)
    
    # wav.scp, text_char, text_kana, text_phone 각각에 동일한 처리를 실시한다
    for filename in ['wav.scp', 'text_char', 
                     'text_kana', 'text_phone']:
        # 모든 데이터 read 모드에서 / 평가(test) / 개발(검증) / 학습 데이터 목록을 쓰기 모드로 열기
        with open(os.path.join(all_dir, filename), #UTF8을 적어줘야 함
                  mode='r',encoding='UTF8') as all_file, \
                  open(os.path.join(out_eval_dir, filename), 
                  mode='w',encoding='UTF8') as eval_file, \
                  open(os.path.join(out_dev_dir, filename), 
                  mode='w',encoding='UTF8') as dev_file, \
                  open(os.path.join(out_train_small_dir, filename), 
                  mode='w',encoding='UTF8') as train_small_file, \
                  open(os.path.join(out_train_large_dir, filename), 
                  mode='w',encoding='UTF8') as train_large_file:
            # 한 줄씩 읽고 평가 / 개발 / 학습 데이터 목록에 씁니다.
            for i, line in enumerate(all_file):
                if i < 250:
                    # 1~250: 평가(test) 데이터 목록에 쓰기
                    eval_file.write(line)
                elif i < 500:
                    # 251~500: 개발(검증) 데이터 목록에 쓰기
                    dev_file.write(line)
                else:
                    # 501~5000: 학습(대) 데이터 리스트에 쓰기
                    train_large_file.write(line)
                    if i < 1500:
                        # 501~1500: 학습(소) 데이터 목록에 쓰기
                        train_small_file.write(line)
